In [1]:
#In the begining, ther is time
import time

In [2]:
#Quality of life
#Some things we do generate a lot of warnings, and it just becomes clutter.
import warnings
warnings.filterwarnings("ignore")
#I like it when my notebook helps me out
%config IPCompleter.greedy=True
#Sometiems you just need to print pretty
from pprint import pprint
#Lots of these operations take many minutes to complete. So it behooves us to pickle the outputs and just unpickle them each time we re-open the notebook
import pickle


Data is from: https://www.kaggle.com/datasets/toygarr/datasets-for-natural-language-processing

Dataset is from a collection of sentiment datasets, but we just want to play with the food one for now.

"ctweet, stweet, food" datasets are positive or negative analysis (sentiment) -> 0 negative -> 1 positive (ctweet has neutral 0, 1, 2)

we're assuming that any code we write to deal with this food data set will be extensible later by simply adding the rest of the data if we so choose. At the moment it's simply faster to work with a smaller subset of the data as we design the pipeline and it's displays.

In [3]:
import pandas as pd
import nltk
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk import ngrams
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
import seaborn as sns

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import DBSCAN
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
import os

In [5]:
#collocation tools
import collections
from nltk.collocations import *

### Load in the dataset and do some basic confirmations

In [6]:
df= pickle.load(open('built_out_df', 'rb'))

In [7]:
df.shape

(363219, 8)

In [8]:
df.head(2)

,text,Y,tokenized,nostops,stemmed,bi_grams,tri_grams,pos_tagged
0,i was put off at first by the green powder but...,1,"[i, was, put, off, at, first, by, the, green, ...","[put, first, green, powder, bad, little, grain...","[put, first, green, powder, bad, littl, graini...","[(put, first), (first, green), (green, powder)...","[(put, first, green), (first, green, powder), ...","[(i, NN), (was, VBD), (put, VBN), (off, RP), (..."
1,these ginger chews are too good to be true i t...,1,"[these, ginger, chews, are, too, good, to, be,...","[ginger, chews, good, true, try, limit, one, d...","[ginger, chew, good, true, tri, limit, one, da...","[(ginger, chew), (chew, good), (good, true), (...","[(ginger, chew, good), (chew, good, true), (go...","[(these, DT), (ginger, JJR), (chews, NNS), (ar..."


All of the below is commented out, because it's the process I used, but now it's all been pickled and I just load the pickle.

In [9]:
#test = pd.read_csv('C:/Users/Prathmun/Documents/Springboard Jupyter/Sentiment-Capstone/data/food/test.csv')

In [10]:
#test.shape

In [11]:
#test.head()

In [12]:
#train = pd.read_csv('C:/Users/Prathmun/Documents/Springboard Jupyter/Sentiment-Capstone/data/food/train.csv')

In [13]:
#train.shape

In [14]:
#train.head()

In [15]:
#frames = [train, test]

### Combine the test/train sets as we're going to do our own splits later

In [16]:
#df = pd.concat(frames)

In [17]:
#df.shape

In [18]:
#df.value_counts()

In [19]:
stopwords = nltk.corpus.stopwords.words("english")

### Tokenize text

In [20]:
#df['tokenized'] = df.text.apply(word_tokenize)



In [21]:
#df.head(2)

### Remove Stop Words


In [22]:
#df['nostops'] = df.tokenized.apply(lambda  x: [item for item in x if item not in stopwords])

In [23]:
# df.head(2)

### Stemming 
Might do lemmatization later, but stemming is simpler and more reliable and we're looking to get something working before we refine it. Lemmatization is a step we can experiment with in our refinement stages

We chose snowball because it's an older, and stable stemmer that incorporates improvements from the older stemmer algorithm Porter's real world experience
Snowball docs: https://www.nltk.org/api/nltk.stem.snowball.html
Article that informed our decision to choose Snowball: https://towardsdatascience.com/stemming-lemmatization-what-ba782b7c0bd8


In [24]:
# stemmer = SnowballStemmer("english")
# df['stemmed'] = df.nostops.apply(lambda x: [stemmer.stem(item) for item in x])

In [25]:
#df.head(1)

In [26]:
#df.stemmed.head(2)

### Bi-gram and n-gram manufacture

It looks as if stemming can increase recall, even in short texts but can also cause problems. We're going to move forward with stemmed words for now, but again can return to this if we seek optimization tasks down the line
Source: https://stackoverflow.com/questions/47219389/compute-word-n-grams-on-original-text-or-after-lemma-stemming-process#:~:text=Computing%20word%20n%2Dgrams%20after,you%20want%20to%20do%20it.

In [27]:
#df['bi_grams'] = df.stemmed.apply(lambda y: [x for x in ngrams(y, 2)])

In [28]:
#df.bi_grams.head(2)

In [29]:
#df['tri_grams'] = df.stemmed.apply(lambda y: [x for x in ngrams(y, 3)])

In [30]:
#df.tri_grams.head(2)

## Part of speach tagging
https://www.nltk.org/book/ch05.html

In [31]:
#df['pos_tagged'] = df.tokenized.apply(nltk.pos_tag)

In [32]:
#df['pos_tagged'].head()

I did this kind of just to see what it would be like. It's not clear that this is going to be useful going forward.

## Vectorizing the corpus
#Creating a df for each degree of n-gram vector, for easy organization
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [33]:
#Vectorizer_Agent_mono = CountVectorizer(preprocessor=None, stop_words=stopwords)

In [34]:
#vectored_no_stops_mono = Vectorizer_Agent_mono.fit_transform(df.text.values) 

In [35]:
vectored_no_stops_mono = pickle.load(open("vectored_no_stops_mono", 'rb'))

In [36]:
type(vectored_no_stops_mono)

scipy.sparse.csr.csr_matrix

In [37]:
#pickle.dump(vectored_no_stops_mono, open("vectored_no_stops_mono", 'wb'))

### Bigram Vector

In [38]:
#Vectorizer_Agent_bi = CountVectorizer(preprocessor=None, stop_words=stopwords, ngram_range=(1,2))

In [39]:
#vectored_no_stops_bi = Vectorizer_Agent_bi.fit_transform(df.text.values) 

In [40]:
vectored_no_stops_bi = pickle.load(open("vectored_no_stops_bi", 'rb'))

In [41]:
type(vectored_no_stops_bi)

scipy.sparse.csr.csr_matrix

In [42]:
#pickle.dump(vectored_no_stops_bi, open("vectored_no_stops_bi", 'wb'))

### Trigram Vector

In [43]:
#Vectorizer_Agent_tri = CountVectorizer(preprocessor=None, stop_words=stopwords, ngram_range=(1,3))

In [44]:
#vectored_no_stops_tri = Vectorizer_Agent_tri.fit_transform(df.text.values) 

In [45]:
vectored_no_stops_tri = pickle.load(open("vectored_no_stops_tri", 'rb'))

In [46]:
type(vectored_no_stops_tri)

scipy.sparse.csr.csr_matrix

In [47]:
#pickle.dump(vectored_no_stops_tri, open("vectored_no_stops_tri", 'wb'))

## Word Clouds/bar charts
### divided by sentiment lables

Make giant strings for each set of words. Total, negative sentiment, and positive sentiment

In [48]:
start = time.time()
nostops_text_no_mods = " ".join(" ".join(listo) for listo in df.nostops)
stop = time.time()
print(str(stop-start))

3.1830010414123535


In [49]:
nostops_text_pos = " ".join(" ".join(listo) for listo in df.nostops[df['Y'] == 1])

In [50]:
nostops_text_neg = " ".join(" ".join(listo) for listo in df.nostops[df['Y'] == 0])

In [51]:
def word_cloud_preprocessing(text):
    #I just love knowing how long my functions take to execute
    start = time.time()
    total_text_word_count = Counter(word_tokenize(text))
#     for stop in stopwords:
#         if stop in total_text_word_count.keys():
#             del total_text_word_count[stop]
    count_dict = dict(total_text_word_count, index=(len(total_text_word_count)))
    series_for_heads = pd.Series(count_dict)
    x = series_for_heads.sort_values(ascending=False).head(15).index
    y = series_for_heads.sort_values(ascending=False).head(15)
    stop = time.time()
    print(str(stop-start))
    #build the actual wordcloud boject
    total_wordcloud = WordCloud(max_font_size=50, max_words=25, background_color="black",
                               width=400,
                                height=400,
                                relative_scaling=1,
                                colormap='winter',
                                collocations=False,
                                stopwords=stopwords,
                                scale=2
                            
                               ).generate(text)
    stop = time.time()
    print(str(stop-start))
    return x, y, total_wordcloud

In [52]:
def word_cloud(x, y, wordcloud):
    # Display the generated image:
    fig, axes = plt.subplots(1, 2, figsize=(15,5))
    #testing setting figsize on subfigure directly, rather than at this higher level
    #sns.set(rc = {'figure.figsize':(40,5)})
    
    axes[0].set_title('Top Word counts')
    sns.barplot(x, y, ax=axes[0], palette="winter")
    axes[0].tick_params(axis='x', rotation=45)
    
    axes[1].imshow(wordcloud, interpolation='bilinear')
    axes[1].axis("off")
    axes[1].set_title('Word Cloud')
    plt.show()

In [53]:
def bar_and_cloud_generator(text):
    bar_x, bar_y, cloud_object = word_cloud_preprocessing(text)
    word_cloud(bar_x, bar_y, cloud_object)

## All words

50.12713050842285
66.56471300125122


In [54]:
bar_and_cloud_generator(nostops_text_no_mods)

# Words from sentances labeled positive

All the way to product this is consistent to the total word count

42.25355911254883
55.96883845329285


In [55]:
bar_and_cloud_generator(nostops_text_pos)

# Words from sentances labeled negative

8.628087520599365
11.424883365631104


In [56]:
bar_and_cloud_generator(nostops_text_neg)

We can see there's considerable overlap between all three ways of organizing our corpus.

# n-gram/colocation analysis

https://www.nltk.org/howto/collocations.html

https://towardsdatascience.com/generating-colocations-n-grams-6dd8bea31d13

Documentation for how the association scores work
https://nlp.stanford.edu/fsnlp/promo/colloc.pdf

### All n-grams

In [57]:
start = time.time()
prototype_string = " ".join(df.text.sample(5))
stop = time.time()
print(str(stop-start))

0.01899409294128418


In [58]:
start = time.time()
big_string = " ".join(df.text.values)
stop = time.time()
print(str(stop-start))

0.1507117748260498


In [59]:
len(prototype_string)

3272

In [60]:
start = time.time()
tri_finder = TrigramCollocationFinder.from_words(word_tokenize(prototype_string))
stop = time.time()
print(str(stop-start))

0.011001825332641602


In [61]:
trigram_measures = nltk.collocations.TrigramAssocMeasures()
tri_finder.nbest(trigram_measures.pmi, 10)



[('anything', 'despite', 'having'),
 ('bought', 'at', 'stores'),
 ('commonly', 'bought', 'at'),
 ('contains', 'pvc', 'particles'),
 ('dream', 'ice', 'cream'),
 ('feeling', 'poorly', 'afterwards'),
 ('grilled', 'cheese', 'though'),
 ('its', 'side', 'rather'),
 ('limit', 'refined', 'sugars'),
 ('refined', 'sugars', 'wheat')]

In [62]:
tri_finder.apply_freq_filter(3)
type(tri_finder.nbest(trigram_measures.pmi, 10))

list

In [63]:
def bi_sorter(text,bi_freq=4,tri_freq=3):
    bi_start = time.time()
    #Tokenizes the string and sets up a finder
    bi_finder = BigramCollocationFinder.from_words(word_tokenize(text))
    #Initiates the scorer
    bigram_measures = nltk.collocations.BigramAssocMeasures()
    #Culls infrequent bigrams
    bi_finder.apply_freq_filter(bi_freq)
    #finds likelihood ratios
    bi_likilhood_ratios = bi_finder.score_ngrams(bigram_measures.likelihood_ratio)
    #makes dict to land keys/scores in
    bi_assoc_measures = collections.defaultdict(list)
    #adds keys/scores to dict
    for key, scores in bi_likilhood_ratios:
       bi_assoc_measures[key[0]].append((key[1], scores))

    # Sort keyed bigrams by strongest association.                                  
    for key in bi_assoc_measures:
       bi_assoc_measures[key].sort(key = lambda x: -x[1])
    
    bi_end = time.time()
  
    print('bigram dictionary took ' + str(bi_end-bi_start) + " seconds to build")

    
    return bi_assoc_measures

In [64]:
#This works, but builds an absolutely massive object, and takes an absurdely long time to build. May use it later, but for now
#it's not vital and I'd prefer not to be slowed down quite so much
def tri_sorter(text, tri_freq=3):
    tri_start = time.time()
    #Same thing but for trigrams!
     #Tokenizes the string and sets up a finder
    tri_finder = TrigramCollocationFinder.from_words(word_tokenize(text))
    #Initiates the scorer
    trigram_measures = nltk.collocations.TrigramAssocMeasures()
    #Culls infrequent bigrams
    tri_finder.apply_freq_filter(tri_freq)

    #finds likelihood ratios
    tri_likilhood_ratios = tri_finder.score_ngrams(trigram_measures.likelihood_ratio)
    #makes dict to land keys/scores in
    tri_assoc_measures = collections.defaultdict(list)
    #adds keys/scores to dict
    for key, scores in bi_likilhood_ratios:
       tri_assoc_measures[key[0]].append((key[1], scores))

    # Sort keyed trigrams by strongest association.                                  
    for key in tri_assoc_measures:
       tri_assoc_measures[key].sort(key = lambda x: -x[1])
    tri_end = time.time()
    print('trigram dictionary took ' + str(tri_end-tri_start) + " seconds to build")
    return tri_assoc_measures

In [65]:
bigram_association_measures= bi_sorter(big_string)


bigram dictionary took 142.13208150863647 seconds to build


In [66]:
#trigram_association_measures = tri_sorter(prototype_string)

In [67]:
#pickle.dump(bigram_association_measures, open( "bigram_association_measures", "wb" ) )

In [68]:
bigram_association_measures = pickle.load(open( "bigram_association_measures", "rb" ) )

In [69]:
#pickle.dump(trigram_association_measures, open( "trigram_association_measures", "wb" ) )

In [70]:

print( 'coffee', bigram_association_measures['coffee'][:5])
print( 'good', bigram_association_measures['good'][:5])
print( 'taste', bigram_association_measures['taste'][:5])

coffee [('maker', 17514.065582340292), ('drinker', 9753.677792074923), ('shop', 5026.192223860347), ('is', 4295.8953869268835), ('i', 3513.2238387233565)]
good [('for', 16115.028301977072), ('as', 12363.535222527982), ('quality', 9527.692492398332), ('deal', 9457.073531498123), ('price', 8945.233465605977)]
taste [('like', 24251.766103757236), ('buds', 16954.914011210938), ('of', 13706.92253242369), ('great', 8397.034071352658), ('is', 7834.87312438912)]


### Postiive Text Collocations

In [71]:
text_pos = " ".join(" ".join(listo) for listo in df.text[df['Y'] == 1])

In [72]:
pos_bigram_association_measures= bi_sorter(text_pos)


bigram dictionary took 263.3205075263977 seconds to build


In [73]:
#pickle.dump(pos_bigram_association_measures, open( "pos_bigram_association_measures", "wb" ) )

### Negative Text Collocations

In [74]:
text_neg = " ".join(" ".join(listo) for listo in df.text[df['Y'] == 0])

In [75]:
neg_bigram_association_measures= bi_sorter(text_neg)


bigram dictionary took 52.21445989608765 seconds to build


In [76]:
#pickle.dump(neg_bigram_association_measures, open( "neg_bigram_association_measures", "wb" ) )

# DF Pickle Test

In [77]:
#pickle.dump(df, open( "built_out_df", "wb" ) )

## Clustering and Secondary EDA

We played briefly with using DBSCAN, but it didn't converge at all, it gave us 360k categories

Thinking that instead of conventional clustering we're going to play with latent dirichlet allocation because it'll do a similar thing as K-means, but also provide us with a lot more information about the composition of those clusters.
Original Paper: https://ai.stanford.edu/~ang/papers/nips01-lda.pdf
Tutorial we used to help us get going: https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0
Docs for the gensim module we're sdoing the lda with: https://radimrehurek.com/gensim/models/ldamodel.html

In [78]:
#Tools for lDA visualization

#Normally I like to my imports at the top of a ntoebook, but these imports make the
#notebook spam an annoying warning that I can only fix by downgrading some stuff, which I am disinclined to do.
import pyLDAvis.gensim_models 
import pickle 
import pyLDAvis

In [79]:
#stemmed_texts = list(df.stemmed)

c:\users\prathmun\appdata\local\programs\python\python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [80]:
#makes a dictionary mapping to unique IDs
#id2word = Dictionary(stemmed_texts)


c:\users\prathmun\appdata\local\programs\python\python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [81]:
#makes a bag of words version of the Gensim Dictionary
#corpus = [id2word.doc2bow(text) for text in stemmed_texts]


c:\users\prathmun\appdata\local\programs\python\python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [84]:
# # Train the model on the corpus.
num_topics = 10
# lda = LdaModel(corpus,  id2word=id2word, num_topics=num_topics)

c:\users\prathmun\appdata\local\programs\python\python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [85]:
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself

if 1 == 0:
    LDAvis_prepared = pyLDAvis.gensim_models .prepare(lda, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
        
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, 'ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

c:\users\prathmun\appdata\local\programs\python\python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.034156  0.102538       1        1  18.701455
0      0.001260  0.153829       2        1  12.499609
3      0.082462  0.050674       3        1  12.429818
6      0.060895  0.009963       4        1  12.205238
5      0.070216 -0.038195       5        1  11.090821
1      0.106242 -0.016197       6        1   8.957325
9      0.054929 -0.112165       7        1   8.059191
8     -0.210783  0.118824       8        1   6.103188
4      0.156146 -0.097449       9        1   5.076460
7     -0.287210 -0.171823      10        1   4.876894, topic_info=        Term          Freq          Total Category  logprob  loglift
492      tea  99003.000000   99003.000000  Default  30.0000  30.0000
149    coffe  93715.000000   93715.000000  Default  29.0000  29.0000
289     food  79496.000000   79496.000000  Default  28.0000  28.0000
203      dog  51436.000000   51436.000000  Default  27.0000  27.0000
371   amazon  60278.000000   60278.000000  Default  26.0000  26.0000
...      ...           ...            ...      ...      ...      ...
21     natur   3431.434983   22783.524441  Topic10  -5.2791   1.1276
1423   pasta   2956.803355   11966.995868  Topic10  -5.4280   1.6226
36       use   3475.458061  120604.910709  Topic10  -5.2664  -0.5261
735     look   3116.293078   40026.729117  Topic10  -5.3754   0.4678
743   packag   2954.797903   36570.060921  Topic10  -5.4287   0.5049

[736 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
665       3  0.040569      add
665       4  0.137652      add
665       5  0.456680      add
665       6  0.026131      add
665       7  0.289021      add
...     ...       ...      ...
5308      9  0.999183     yogi
698       3  0.275255   yogurt
698       7  0.724453   yogurt
3821      8  0.998955  younger
6037      6  0.999380     zico

[1960 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 4, 7, 6, 2, 10, 9, 5, 8])

This is pretty neat, but honestly I don't care very much. These distributions don't really affect what I'm about to do. I was just curious to try this.

We might do something similar to the earlier word clouds and break this up by sentiment. SEe if the clusters are significantly different across sentiments, but again it's not deeply relevant to our long term goals, so we may not take the time